In [5]:
#!pip install catboost
#!pip install xgboost

In [6]:
#!pip install lightgbm

In [7]:

import numpy as np
import pandas as pd

df = pd.read_csv('/content/LE.csv')
df = df [['Country','Year','Status','Life expectancy']]

In [8]:
df.head()

,Country,Year,Status,Life expectancy
0,Afghanistan,2015,Developing,65.0
1,Afghanistan,2014,Developing,59.9
2,Afghanistan,2013,Developing,59.9
3,Afghanistan,2012,Developing,59.5
4,Afghanistan,2011,Developing,59.2


In [10]:
!pip install catboost
from catboost import CatBoostRegressor, Pool # Importing immediately after install

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 7.2 MB/s eta 0:00:00


In [11]:
from catboost import CatBoostRegressor, Pool
X = df.drop(columns='Life expectancy')
y = df['Life expectancy']

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20,
                                                    random_state=1)

pool_train = Pool(X_train, y_train,
                  cat_features = ['Country','Year','Status'])

pool_test = Pool(X_test, cat_features = ['Country','Year','Status'])


#CatBoost

import time

start = time.time()

cbr = CatBoostRegressor(iterations=100,max_depth=2)

cbr.fit(pool_train)
y_pred = cbr.predict(X_test)

from sklearn.metrics import r2_score as RSquared


cb_rsquared = np.sqrt(RSquared(y_test, y_pred))
print("R Squared for CatBoost: ", np.mean(cb_rsquared))

end = time.time()
diff = end - start
print('Execution time:', diff)


Learning rate set to 0.304491
0:	learn: 7.9652996	total: 50.7ms	remaining: 5.02s
1:	learn: 6.8750145	total: 51.5ms	remaining: 2.52s
2:	learn: 6.3649201	total: 54.8ms	remaining: 1.77s
3:	learn: 6.0458584	total: 55.9ms	remaining: 1.34s
4:	learn: 5.7968748	total: 57.5ms	remaining: 1.09s
5:	learn: 5.6849516	total: 61ms	remaining: 955ms
6:	learn: 5.5930909	total: 61.8ms	remaining: 822ms
7:	learn: 5.5501911	total: 77.5ms	remaining: 891ms
8:	learn: 5.5157724	total: 78.4ms	remaining: 793ms
9:	learn: 5.4922729	total: 84.3ms	remaining: 758ms
10:	learn: 5.4620773	total: 90.1ms	remaining: 729ms
11:	learn: 5.4609070	total: 94.2ms	remaining: 691ms
12:	learn: 5.4574215	total: 100ms	remaining: 669ms
13:	learn: 5.4314251	total: 104ms	remaining: 636ms
14:	learn: 5.4191324	total: 106ms	remaining: 600ms
15:	learn: 5.4029769	total: 107ms	remaining: 564ms
16:	learn: 5.3918803	total: 109ms	remaining: 532ms
17:	learn: 5.3861049	total: 110ms	remaining: 503ms
18:	learn: 5.3766558	total: 112ms	remaining: 476ms
1

In [12]:
#XGBoost
import xgboost as xgb
from sklearn import preprocessing
X = df.drop(columns='Life expectancy')
y = df['Life expectancy']

lbl = preprocessing.LabelEncoder()
#Country','Year','Status
X['Country'] = lbl.fit_transform(X['Country'].astype(str))
X['Year'] = lbl.fit_transform(X['Year'].astype(str))
X['Status'] = lbl.fit_transform(X['Status'].astype(str))

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20,
                                                    random_state=1)

start = time.time()
#X_train["Species"].astype("category")
xgbr = xgb.XGBRegressor()

xgbr.fit(X_train, y_train)
y_pred = xgbr.predict(X_test)

xgb_rmse = np.sqrt(RSquared(y_test, y_pred))
print("R Squared for XGBoost: ", np.mean(xgb_rmse))

end = time.time()
diff = end - start
print('Execution time:', diff)

R Squared for XGBoost:  0.9790585421042686
Execution time: 0.794001579284668


In [13]:
import lightgbm
X = df.drop(columns='Life expectancy')
y = df['Life expectancy']
obj_feat = list(X.loc[:, X.dtypes == 'object'].columns.values)

for feature in obj_feat:
    X[feature] = pd.Series(X[feature], dtype="category")

start = time.time()

lgbmr = lightgbm.LGBMRegressor()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20,
                                                    random_state=1)

lgbmr.fit(X_train, y_train)
y_pred = lgbmr.predict(X_test)

lgbm_rsquared = np.sqrt(RSquared(y_test, y_pred))
print()
print("R squared for LightGBM: ", np.mean(lgbm_rsquared))

end = time.time()
diff = end - start
print('Execution time:', diff)

/usr/local/lib/python3.11/dist-packages/dask/dataframe/__init__.py:42: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)


[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000822 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 203
[LightGBM] [Info] Number of data points in the train set: 2342, number of used features: 3
[LightGBM] [Info] Start training from score 69.262724

R squared for LightGBM:  0.9767455250796998
Execution time: 0.4577016830444336
